In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
df = pd.read_csv('test_subset.csv')
df

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner m...,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben'...",...,...
1,ekaterini@gmail.com,833,Der Klimawandel ist ein völlig normaler Prozes...,positive,...,...,völlig normaler Prozess,Klimawandel
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialo...,negative,Demokraten,"['missliebige Partei ', 'Was denken die']",...,...
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten währen...",negative,Corona,die während Corona auch gern auf die Maske ver...,...,...
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahne...,positive,...,...,handelt moralisch richtig,"['Israel ', 'Israel ']"
...,...,...,...,...,...,...,...,...
295,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Ziv...,negative,Israel,"['begeht Kriegsverbrechen', 'aber Kritik an Is...",...,...
296,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,...,geht's noch?,...,...
297,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise...",negative,Klimakrise,"['hypokapitalistischen', 'Nice one, Humanity',...",...,...
298,ekaterini@gmail.com,473,Kleine Nachhilfe: Ein paar hunderttausend Grün...,negative,Grüne,eine Klatsche bekommen habt,...,...


In [6]:
texts = list(df.text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-1.1-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

#prompt = f"Classify the sentiment of each word in the texts into ONE of the three classes: neutral, negative or positive. Split the answer in two columns: Token and Label. Text: "
prompt = f"Classify the sentiment of each word in the texts into ONE of the three classes: neutral, negative or positive. Split the answer in a dictionary with two keys:Words and Labels. Text: "

answers = []
#text_list =[ "Ukraine deckt auf 😂😂😂😂😂\nDie mussten was sagen, nicht das man denkt der Staat wäre korrupt und es würde keinen Kampf dagegen geben ☝️\n\nKorrupt bleibt korrupt - auch die, da wird sich auch nichts änder 😆",'Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund']
for text in texts:
    chat_template = [{"role" : "user", "content" : f"{prompt} {text}"}]

    chat_prompt = tokenizer.apply_chat_template(chat_template, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(chat_prompt, add_special_tokens=False, return_tensors="pt", max_length=512)
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens = 512)
    answer = tokenizer.decode(outputs[0])
    answers.append(answer)
    #print(answer)
print(len(answers))

In [ ]:
#Processing the outputs of above results, to generate a list of rows with 'tokens and labels'
rows = []
for answer in answers:
    text = answer.split('<start_of_turn>model')[0].split('Text:')[1] 
    if len(answer.split('<start_of_turn>model')[1].split('------- | --------')) > 1:
        tokens = []
        labels = []
        for split_answer in answer.split('<start_of_turn>model')[1].split('------- | --------')[1].split('\n'):
            #print('split after split',split_answer.split('|'))
            split_after = split_answer.split('|')
            #print(split_after[0])
            for index,item in enumerate(split_after):
                #print(item)
                if item!= '':
                    if index %2 ==0:
                        tokens.append(item)
                    else:
                        labels.append(item.lower())
   
        rows.append({'tokens':tokens,'labels':labels})
    else:
        print(answer)
      
print(len(rows))
#print(rows)

In [25]:
#Removing the entries from df for which the answers by LLM were not outputted in the correct format
df= df[(df['text'] != 'Da haben ja die Grünen nichts zu befürchten 😉wo nichts ist kann auch nichts angegriffen werden 👍')]
df= df[(df['text'] != 'AfD: "Wir stehen für das christliche Abendland!" Deutsche Kirchen: "The fuck you are!"')]
df=df.reset_index(drop=True)
df

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner m...,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben'...",...,...
1,ekaterini@gmail.com,833,Der Klimawandel ist ein völlig normaler Prozes...,positive,...,...,völlig normaler Prozess,Klimawandel
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialo...,negative,Demokraten,"['missliebige Partei ', 'Was denken die']",...,...
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten währen...",negative,Corona,die während Corona auch gern auf die Maske ver...,...,...
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahne...,positive,...,...,handelt moralisch richtig,"['Israel ', 'Israel ']"
...,...,...,...,...,...,...,...,...
293,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Ziv...,negative,Israel,"['begeht Kriegsverbrechen', 'aber Kritik an Is...",...,...
294,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,...,geht's noch?,...,...
295,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise...",negative,Klimakrise,"['hypokapitalistischen', 'Nice one, Humanity',...",...,...
296,ekaterini@gmail.com,473,Kleine Nachhilfe: Ein paar hunderttausend Grün...,negative,Grüne,eine Klatsche bekommen habt,...,...


In [ ]:
#Adding the above generated tokens and labels to our dataframe and saving it for future reference
df['token_and_labels_FromGemmaLLM'] = rows
df.to_csv('Results_From_Google_withTokensAndLabels.csv', index = False)

In [2]:
#You can either read the csv or just use our dataframe df
df = pd.read_csv('Results_From_Google_withTokensAndLabels.csv', converters = {'token_and_labels_FromGemmaLLM':literal_eval} )

In [3]:
#pre-processing the data so that the spans are not strings but lists
df.rename(columns={'negative object_spans':'negative_object_spans','negative spans':'negative_spans','positive object_spans':'positive_object_spans','positive spans':'positive_spans'}, inplace=True)
new_list_positive_object_spans = []
new_list_positive_spans = []
new_list_negative_object_spans = []
new_list_negative_spans =[]

list_positive_object_spans = df.positive_object_spans
list_positive_spans = list(df.positive_spans)
list_negative_object_spans = df.negative_object_spans
list_negative_spans = list(df.negative_spans)

#print(list_negative_object_spans)
#print(list_negative_spans)
#print(list_positive_object_spans)

for item in list_positive_object_spans:
    if not '[' in item:
        new_list_positive_object_spans.append(item.split())
    else:
        new_list_positive_object_spans.append(item)

for item in list_positive_spans:
    if not '[' in item:
        new_list_positive_spans.append(item.split())
    else:
        new_list_positive_spans.append(item)

for item in list_negative_object_spans:
    if not '[' in item:
        new_list_negative_object_spans.append(item.split())
    else:
        new_list_negative_object_spans.append(item)

for item in list_negative_spans:
    if not '[' in item:
        new_list_negative_spans.append(item.split())
    else:
        new_list_negative_spans.append(item)

#print(new_list_positive_object_spans)

In [ ]:
df['negative_object_spans'] = new_list_negative_object_spans
df['negative_spans'] = new_list_negative_spans
df['positive_object_spans'] = new_list_positive_object_spans
df['positive_spans'] = new_list_positive_spans

In [15]:
#You can see the spans are all lists now
pd.options.display.max_colwidth = 1000
df

,user,instance_id,text,sentiment,negative_object_spans,negative_spans,positive_spans,positive_object_spans,token_and_labels_FromGemmaLLM
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben', ' 😕 ']",[...],[...],"{'tokens': ['Politiker ', 'Krieg ', 'Bock ', 'Drehen ', 'Spieß ', 'Frage ', 'Freund '], 'labels': [' negative', ' negative', ' negative', ' negative', ' negative', ' neutral', ' neutral<eos>']}"
1,ekaterini@gmail.com,833,"Der Klimawandel ist ein völlig normaler Prozess, den der Mensch nicht beeinflussen kann.\nChange my Mind",positive,[...],[...],"[völlig, normaler, Prozess]",[Klimawandel],"{'tokens': ['Klimawandel ', 'normaler ', 'prozesse ', 'Mensch ', 'beeinflussen '], 'labels': [' negative', ' neutral', ' neutral', ' neutral', ' negative<eos>']}"
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialog nicht an eine missliebige Partei vermieten. \n\nWas denken die „#Demokraten“ damit bezwecken zu wollen?,negative,[Demokraten],"['missliebige Partei ', 'Was denken die']",[...],[...],"{'tokens': ['Man ', 'stirbt ', 'Stadthalle ', 'für ', 'einen ', 'Bürgerdialog ', 'nicht ', 'an ', 'missliebige ', 'Partei ', 'vermieten ', 'Was ', 'denken ', 'die ', 'Demokraten ', 'mit ', 'unsbwecken '], 'labels': [' neutral', ' negative', ' neutral', ' neutral', ' neutral', ' positive', ' negative', ' neutral', ' negative', ' neutral', ' negative', ' neutral', ' neutral', ' neutral', ' positive', ' neutral', ' positive<eos>']}"
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten während Corona auf so viel verzichten"". Der Satz kommt bestimmt von denen, die während Corona auch gern auf die Maske verzichtet haben.",negative,[Corona],"[die, während, Corona, auch, gern, auf, die, Maske, verzichtet, haben.]",[...],[...],"{'tokens': ['Besonders ', 'popular ', 'mussen ', 'während ', 'Corona ', 'verzicht '], 'labels': [' positive', ' positive', ' neutral', ' neutral', ' negative', ' negative<eos>']}"
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahnen.\nIsrael handelt moralisch richtig.,positive,[...],[...],"[handelt, moralisch, richtig]","['Israel ', 'Israel ']","{'tokens': ['Die USA ', 'söllten ', 'mäßigen ', 'Israel ', 'handelt ', 'moralisch ', 'richtig '], 'labels': [' negative', ' neutral', ' negative', ' positive', ' neutral', ' positive', ' positive<eos>']}"
...,...,...,...,...,...,...,...,...,...
293,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Zivilbevölkerung im Gaza Streifen aber Kritik an Israel will niemand hören,negative,[Israel],"['begeht Kriegsverbrechen', 'aber Kritik an Israel will niemand hören']",[...],[...],"{'tokens': ['Israel ', 'Kriegsverbrechen ', 'Zivilbevölkerung ', 'Kritik ', 'Will '], 'labels': [' negative', ' negative', ' neutral', ' negative', ' negative<eos>']}"
294,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,[...],"[geht's, noch?]",[...],[...],"{'tokens': ['Sogenannte ', 'Zivilisten? ', 'geht's ', 'noch? '], 'labels': [' neutral', ' neutral', ' positive', ' negative<eos>']}"
295,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise soviel Energie, Zeit und Geld in die Entwicklung einer hypokapitalistischen, fahrenden Kleinstadt auf dem Meer investieren. Ich könnte mir wirklich nichts vorstellen, wo diese Ressourcen besser zu gebrauchen wären! Nice one, Humanity! 💸",negative,[Klimakrise],"['hypokapitalistischen', 'Nice one, Humanity', ' 💸']",[...],[...],"{'tokens': ['normal ', 'klimakrise ', 'Energie ', 'Zeit ', 'Geld ', 'hypokapitalistisch ', 'fahrende ', 'Kleinstadt ', 'Meer ', 'resourcen ', 'brauchen ', 'nice ', 'one ', 'Humanity ', '💸 '], 'labels': [' neutral', ' negative', ' neutral', ' neutral', ' neutral', ' negative', ' positive', ' positive', ' positive', ' positive', ' 

### Creating data for NER evaluation , where I check if the tokens are in spans , if yes then add the actual and predicted sentiment, if not then add 'O' as in outer

In [ ]:
predicted_labels = []
actual_labels = []
words_in_spans = []
for i in range(len(df)):
    tokens = df['token_and_labels_FromGemmaLLM'][i]['tokens']
    pred_labels_df = df['token_and_labels_FromGemmaLLM'][i]['labels']
    for index,token in enumerate(tokens):
        if token.strip() in df['negative_object_spans'][i] or token.strip() in df['negative_spans'][i] or token.strip() in df['positive_object_spans'][i] or token.strip() in df['positive_spans'][i]:
            #print(token)
            actual_labels.append(df['sentiment'][i])
            words_in_spans.append(token)
            #for label in pred_labels_df:
            predicted_labels.append(pred_labels_df[index])
        else:
            words_in_spans.append(token)
            actual_labels.append('O')
            predicted_labels.append('O')
    words_in_spans.append(' ')
    predicted_labels.append(' ')
    actual_labels.append(' ')
print(len(predicted_labels))
print(len(actual_labels))
print(len(words_in_spans))



In [ ]:
df_ner_LLM = pd.DataFrame(columns=['tokens','actual','predicted'])
df_ner_LLM['tokens'] = words_in_spans
df_ner_LLM['actual'] = actual_labels
df_ner_LLM['predicted'] = predicted_labels
df_ner_LLM.head(50)

In [10]:
#Making sure that there are no <eos> attached to the predicted sentiments
count=0
for i in range(len(df_ner_LLM)):
    df_ner_LLM.loc[i,'predicted'] = str(df_ner_LLM['predicted'][i]).strip(' ')
    if '<eos>' in df_ner_LLM['predicted'][i]:
        df_ner_LLM.loc[i,'predicted'] = str(df_ner_LLM['predicted'][i])[:-5]
        count = count + 1
print(count)


120


In [13]:
#Making sure to refer neutral as 'O' as in outer if present
for i in range(len(df_ner_LLM)):
    if df_ner_LLM['predicted'][i] == 'neutral':
        df_ner_LLM.loc[i,'predicted'] = 'O'

In [14]:
#saving the results as csv file to perform NER evaluation using conlleval
df_ner_LLM.to_csv('ResultsOFNEREvaluationLLMGemma4thAttempt.csv',header=False,index=False, sep='\t')

#### Below are the results of NER Evaluation
##### python conlleval_perl.py -r < ResultsOFNEREvaluationLLMGemma4thAttempt.csv -- command used for NER evaluation
##### processed 2909 tokens with 1049 phrases; found: 777 phrases; correct: 644.
##### accuracy:  86.08%; precision:  82.88%; recall:  61.39%; FB1:  70.54
         #### negative: precision:  92.08%; recall:  65.35%; FB1:  76.45  631
         ####  positive: precision:  43.15%; recall:  39.38%; FB1:  41.18  146